In [1]:
%run /home/bb/src/python/valkyrie/apps/common/run_notebook_import.py

/media/veracrypt1/python/valkyrie/apps/common/run_notebook_import.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import copy
from itertools import product as cartesian_product

from overrides import overrides
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.nn import functional as F

from valkyrie.tools import *
from valkyrie.ml import modules
from valkyrie.ml.utils import tensor, HyperParameters
from valkyrie.ml import utils as ml_utils
from valkyrie.nibelungen.data import DataMgr, Df2T2

/media/veracrypt1/python/valkyrie/lib/valkyrie/nibelungen/data.py:192: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [4]:
from valkyrie.ml.data import DataModule

sdate, edate = '2023-01-01', '2023-02-28'
instrs = ['BDM.BTC.USDT.FP']
freq = '1s'
ret_n_s = [120]
root_dir = '/home/bb/data/BDM'
#########################################################
#DataMgr
#########################################################
data_mgr = DataMgr(sdate, edate, freq, root_dir, instrs = instrs, ret_n_s = ret_n_s)
df_res = data_mgr.get(instrs[0]) 
channels = ['bpx_last', 'apx_last', 'bq_last', 'aq_last', 'buy_qty_sum', 'sell_qty_sum']
n_channels = len(channels)
df2t2 = Df2T2(df_res, M = 2, xcols = channels, ycol = 'mid_last_ret_120_n')
dm = DataModule.from_dataset(df2t2, None, 2048)

after dropping na 5097417 -> 5097416


In [10]:
np.float32

numpy.float32

In [9]:
df_res.T

,2023-01-01 00:00:05,2023-01-01 00:00:06,2023-01-01 00:00:07,2023-01-01 00:00:08,2023-01-01 00:00:09,2023-01-01 00:00:10,2023-01-01 00:00:11,2023-01-01 00:00:12,2023-01-01 00:00:13,2023-01-01 00:00:14,...,2023-02-28 23:57:50,2023-02-28 23:57:51,2023-02-28 23:57:52,2023-02-28 23:57:53,2023-02-28 23:57:54,2023-02-28 23:57:55,2023-02-28 23:57:56,2023-02-28 23:57:57,2023-02-28 23:57:58,2023-02-28 23:57:59
aq_first,34.209999,34.209999,34.212002,22.304001,38.553001,92.571999,65.972000,54.310001,50.994999,51.213001,...,42.993999,23.621000,17.076000,16.812000,17.597000,18.521000,19.018000,18.183001,15.941000,0.627000
aq_last,34.209999,34.209999,22.069000,38.561001,92.522003,65.892998,58.520000,50.933998,51.213001,46.723999,...,23.621000,17.076000,16.839001,15.979000,18.521000,19.018999,18.650999,15.941000,0.001000,0.811000
aq_max,34.209999,34.209999,35.409000,39.227001,92.584000,95.009003,67.218002,54.664001,51.213001,51.213001,...,43.313999,23.621000,17.091999,16.812000,23.073999,19.020000,19.021000,18.184999,15.966000,2.541000
aq_min,34.209999,34.209999,20.408001,22.304001,38.549999,5.975000,30.471001,26.636000,50.765999,41.973000,...,23.621000,15.544000,16.430000,15.979000,17.597000,18.502001,17.886000,15.941000,0.001000,0.627000
apx_first,16537.599609,16537.599609,16537.599609,16537.599609,16537.599609,16537.000000,16535.900391,16535.400391,16534.400391,16534.400391,...,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23107.099609
apx_last,16537.599609,16537.599609,16537.599609,16537.599609,16537.000000,16535.900391,16535.500000,16534.400391,16534.400391,16534.400391,...,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.800781,23107.099609
apx_max,16537.599609,16537.599609,16537.599609,16537.599609,16537.599609,16537.000000,16535.900391,16535.400391,16534.400391,16534.400391,...,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.800781,23107.099609
apx_min,16537.599609,16537.599609,16537.599609,16537.599609,16537.000000,16535.900391,16535.500000,16534.400391,16534.400391,16534.400391,...,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23106.300781,23107.099609
bpx_first,16537.500000,16537.500000,16537.500000,16537.500000,16537.500000,16536.900391,16535.800781,16535.300781,16534.300781,16534.300781,...,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23107.000000
bpx_last,16537.500000,16537.500000,16537.500000,16537.500000,16536.900391,16535.800781,16535.400391,16534.300781,16534.300781,16534.300781,...,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23106.199219,23106.699219,23107.000000


In [15]:
1 basis point  = 0.01 percetage point

NameError: name 'bps' is not defined

In [14]:
0.000571 * 1e4

5.71

In [13]:
ret_in_bps = -0.000157 * 1e4

-1.5699999999999998

In [ ]:
np.log(y1/y0) = -0.000157

In [11]:
print_df_mem_usage(df_res)

713.6 MB


In [ ]:
class GoogleNet(modules.Regressor):
    def b0(self):
        return nn.BatchNorm2d(self.n_channel);

    def __init__(self, n_channel, lr=0.1, num_classes=10):
        #super(GoogleNet, self).__init__()
        self.n_channel = n_channel
        super(GoogleNet, self).__init__('l1')        
        self.save_hyperparameters()
        self.net = nn.Sequential(self.b0())
        self.net.apply(d2l.init_cnn)

In [ ]:
google_net = GoogleNet(n_channel = n_channels, lr = 0.1)
X = df2t2[0][0]
ml_utils.layer_summary(google_net, X.shape)

In [ ]:
trainer = modules.Trainer(max_epochs=200, num_gpus=1)
with Timer('training'):
    trainer.fit(google_net, dm)

In [ ]:
google_net = google_net.to('cuda:0')
dl = dm.get_dataloader(True)
for i, batch in enumerate(dl):        
    Y = batch[1][:,0].to('cuda:0')
    with torch.no_grad():                        
        Y_hat = google_net(batch[0].to('cuda:0'))
        break

In [ ]:
Y_hat.shape

In [ ]:
Y.shape

In [ ]:
X.shape

In [ ]:
batch_norm_layer = nn.BatchNorm2d(n_channels)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(batch_norm_layer.parameters(), lr=0.01)

In [ ]:
import copy

In [ ]:
num_epochs = 3  # Number of training epochs

for epoch in range(num_epochs):
    for inputs, labels in dm.get_dataloader(True):
        inputs = inputs
        labels = torch.randn_like(inputs)
        optimizer.zero_grad()  # Zero the gradient buffers
        outputs = batch_norm_layer(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [ ]:
outputs = batch_norm_layer(inputs)

In [ ]:
outputs

In [ ]:
inputs

In [ ]:
inputs.std(axis=0)